In [11]:
import random
import math

In [13]:
max = []
levels = 2
numclust = []
dim = 0

In [24]:
def clustergenexec(numdata, numcluster, ranges, dimensions, mindist, lvls):
    global max
    global levels
    global numclust
    global dim

    max = ranges
    levels = lvls
    numclust = numcluster
    dim = dimensions

    centers = []
    data = []
    num = numclust[0]
    newradius = 0
    for i in range(0, len(ranges)):
        newradius += ranges[i] / 2.0
    mindist2 = mindist ** 2
    newmin = mindist2 / (2.0 * (newradius) / len(ranges))
    
    i = 0
    temp = []
    distance = 0

    while i < num:
        temp = []
        for j in range(0, dim):
            temp.append(random.random() * ranges[j])
        centers.append(temp)
        for j in range(0, len(centers) - 1):
            for k in range(0, dim):
                distance += (centers[j][k] - centers[i][k]) ** 2
            if distance < mindist2:
                del centers[i]
                i -= 1
        i += 1
    
    mindistdiv2 = mindist / 2.0
    numperclust = numdata // num
    if levels > 0:
        for i in range(0, num):
            data.append(clustergen(numperclust, mindistdiv2, newmin,
            levels - 1, centers[i]))
    elif levels == 0:
        for i in range(0, num):
                data.append(clusterfromcenter(centers[i], numperclust, mindistdiv2))
                
    return data


In [27]:
def clustergen(numdata, radius, mindist, levels, center):
    global numclust
    num = numclust[len(numclust) - levels - 1]
    temp = []
    mindistdiv2 = mindist / 2.0
    newmin =  mindist ** 2.0 / (2.0 * radius)
    numperclust = numdata // num

    if levels == 0:
        return clusterhelper(numdata, num, radius, mindist, center)
    elif levels != 0:
        centers = centergen(num, radius, mindist, center)
        for i in range(0, num):
            temp += clustergen(numperclust, mindistdiv2, newmin,
            levels - 1, centers[i])
        return temp


In [28]:
def clusterhelper(numdata, numcluster, radius, mindist, center):
    centers = centergen(numcluster, radius, mindist, center)
    numperclust = numdata // numcluster
    mindistdiv2 = mindist / 2.0
    clusters = []

    for i in range(0, numcluster):
        clusters += clusterfromcenter(centers[i], numperclust, mindistdiv2)

    return clusters

In [29]:
def clusterfromcenter(center, numdata, radius):
    data = []
    global xmax
    global ymax
    global zmax
    i = 0
    temp = []
    inbounds = True

    while i < numdata:
        inbounds = True
        temp = gencoord(radius)

        for j in range(0, dim):
            temp[j] += center[j]
            if temp[j] > max[j] or temp[j] < 0:
                inbounds = False
        if inbounds:
            data.append(temp)
            i += 1

    return data

In [18]:
def centergen(numcenter, radius, mindist, refpoint):
    global dim
    global max
    centers = []
    temp = []
    i = 0
    inbounds = True
    mindist2 = mindist ** 2


    while i < numcenter:
        tooclose = False
        inbounds = True
        temp = gencoord(radius/3)
        
        for j in range(0, dim):
            temp[j] += refpoint[j]
            if temp[j] > max[j] or temp[j] < 0:
                inbounds = False

        if  inbounds:
            for j in range(0, i):
                distance = 0
                for k in range(0, dim):
                    distance += (centers[j][k] - temp[k]) ** 2
                if distance < mindist2:
                    tooclose = True
            if not tooclose:
                centers.append(temp)
                i += 1

    return centers

In [32]:
def gencoord(radius):
    global dim
    temp = []
    angle = 0
    anglecos = []
    anglesin = []

    r = random.gauss(0, 1) * radius
    for j in range(0, dim - 2):
        angle = random.random() * math.pi
        anglesin.append(math.sin(angle))
        anglecos.append(math.cos(angle))
    angle = random.random() * 2 * math.pi
    anglesin.append(math.sin(angle))
    anglecos.append(math.cos(angle))
    
    for j in range(0, dim - 1):
        temp.append(r)
        for k in range(0,j):
            temp[j] *= anglesin[k]
        temp[j] *= anglecos[j]
    temp.append(r * anglesin[dim - 2])
    for j in range(0, dim - 1):
        temp[dim - 1] *= anglesin[j]

    return temp

In [33]:
"""dim = 5
print "This is a test coordinate: ", gencoord(5), "\n"
dim = 3
print "This is a test coordinate: ", gencoord(3), "\n"
dim = 10
print "This is a test coordinate: ", gencoord(10), "\n"
dim = 2
print "This is a test coordinate: ", gencoord(0), "\n"
dim = 12
print "This is a test coordinate: ", gencoord(23), "\n\n"

dim = 5
max = [1000,1000,1000,1000,1000]
print "This is 3 test centers: \n", centergen(3, 100, 2, [500,500,500,500,500]), "\n"
dim = 3
max = [500,500,500]
print "This is 5 test centers: \n", centergen(5, 50, 1, [250,250,250]), "\n"
dim = 7
max = [100,100,100,100,100,100,100]
print "This is 2 test centers: \n", centergen(2, 20, 1, [70,70,70,70,70,70,70]), "\n"
dim = 2
max = [300,300]
print "This is 2 test centers: \n", centergen(2, 300, 1, [0,0]), "\n"

dim = 3
max = [1000,1000,1000]
print "This is a test cluster: \n", clusterfromcenter([500,500,500], 5, 20), "\n"
dim = 5
max = [500,500,500,500,500]
print "This is a test cluster: \n", clusterfromcenter([100,100,100,100,100], 4, 25), "\n"
dim = 2
max = [50,50]
print "This is a test cluster: \n", clusterfromcenter([0,0], 6, 10), "\n"

dim = 3
max = [1000,1000,1000]
print "This is a clusterhelper test: \n", clusterhelper(18, 6, 100, 5, [500,500,500]), "\n"
dim = 5
max = [500,500,500,500,500]
print "This is a clusterhelper test: \n", clusterhelper(12, 3, 100, 4, [100,100,100,100,100]), "\n"
dim = 2
max = [100,100]
print "This is a clusterhelper test: \n", clusterhelper(6, 2, 25, 2, [0,0]), "\n"

dim = 3
max = [1000,1000,1000]
numclust = [3,2]
print "This is a clustergen test: \n", clustergen(24, 200, 5, 1, [500,500,500]), "\n"
dim = 5
max = [2000, 2000, 2000, 2000, 2000]
numclust = [3,2,2]
l = clustergen(36, 500, 7, 2, [1000, 1000, 1000, 1000, 1000])
print "This is a clustergen test: \n", l, "\n"

dim = 2
max = [100,100]
numclust = [3,2,2]
l = clustergen(24, 50, 4, 2, [0,0])
print "This is a clustergen test: \n", l, "\n"

"""
l = clustergenexec(36, [3,2,2], [1000, 1000, 1000], 3, 100, 3)
print "This is a clustergenexec test: \n", l, "\n"


This is a clustergenexec test: 
[[[198.53552571118024, 918.8855763258398, 631.8657226585665], [198.47879186840998, 919.0050882359347, 631.8721575349765], [200.24595812647541, 919.4648114294536, 631.0857055976946], [200.57066718881768, 919.3040024566645, 631.0725894895564], [219.46576819368215, 923.2026954391752, 620.9719251610734], [219.5102534638666, 923.1918634902997, 620.9540754646218], [219.84713566184422, 920.7864994804831, 620.9193682137749], [220.05616001434137, 920.76009109142, 620.9051527707195], [220.06653543617352, 913.6680472737687, 628.2503299131087], [220.0161105375943, 913.4838629565721, 628.4427471586883], [217.26047608769994, 913.697055868107, 628.3159308342653], [217.1237081999795, 913.3395484317194, 628.2804919757801]], [[386.5253063258013, 749.0124400249025, 376.69247110441876], [386.6069486799524, 749.2671466072859, 376.6664312046317], [389.21965219358003, 747.4659270829827, 377.3261143531816], [389.4084552569757, 747.5833526619078, 377.3841568577355], [392.2571443